<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=37076cd7a9ea534f7e993f98f3a13f468d8554226dc14727a7011378ec3ffe6d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-04 11:05:14
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.44 C
-------------------
Today PnL: 39.62 K (0.28%)
Current PnL: -24.14 L (-15.6%)
CY Booked + Current PnL: -9.85 L (-6.37%)
-------------------
Total profit:  1.29 L
Total loss:  -25.44 L
-------------------
Total Booked + Current PnL: 17.13 L (13.4%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (9.91%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 88.78 L (61.59%)
Deployed:  1.28 C
Current:  1.44 C
CAGR/XIRR %: 6.84%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.36,-7.43,8.05,0.02,13008.0,-12975.0,161584.0,146.65,69.0,M-SC,15.59,234.0,-1.00,1.16,40.55,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.39,-16.38,19.70,0.09,16600.0,-16511.0,84266.0,95.52,40.0,M-SC,2.77,253.0,-0.99,0.61,10.71,OX40N,NTT,DURABLES
43,ITC,452.00,0.65,-1.51,12.13,10.44,23910.0,-3022.0,197116.0,-41.15,45.0,X-LC,1.57,5.0,-0.13,1.42,3.97,X40,NTT,FMCG
35,ICICIGI,2252.93,0.64,5.91,13.49,20.19,24904.0,10294.0,184614.0,-16.96,48.0,X-MC,6.62,68.0,0.41,1.33,21.91,X40,ATH,INSURANCE
50,MASFIN,398.61,-1.83,-3.80,26.87,22.05,25328.0,-3720.0,94260.0,-16.89,53.0,H-SC,6.91,164.0,-0.15,0.68,37.01,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,3.39,-45.14,217.42,74.15,195156.0,-73845.0,89760.0,-261.57,40.0,H-SC,20.05,149.0,-0.38,0.64,15.06,XR,NTT,CHEMICALS
29,HAPPSTMNDS,1480.71,2.80,-36.20,189.74,84.85,155147.0,-46397.0,81768.0,-24.12,56.0,H-SC,14.00,147.0,-0.30,0.59,4.94,AR,ATH,IT
73,TCS,4389.97,1.75,-12.09,35.68,19.28,108516.0,-41819.0,304137.0,-23.30,70.0,X-LC,6.53,1.0,-0.39,2.19,12.43,X40,ATH,IT
67,SONACOMS,806.63,1.70,-13.50,61.33,39.54,53664.0,-13659.0,87500.0,-32.36,56.0,M-SC,6.26,220.0,-0.25,0.63,23.38,AR,ATH,AUTO
60,REPCOHOME,880.00,1.64,-12.27,116.38,89.82,292511.0,-35164.0,251341.0,-56.04,46.0,H-SC,8.25,160.0,-0.12,1.81,32.98,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,EASEMYTRIP,26.40,-4.63,-56.84,237.60,45.70,181308.0,-100507.0,76308.0,-20.74,52.0,M-SC,23.53,205.0,-0.55,0.55,9.99,XY24,NTT,TRAVEL
51,MEDANTA,1486.00,-1.93,-0.72,24.98,24.08,31483.0,-916.0,126034.0,-8.70,37.0,X-SC,5.96,89.0,-0.03,0.91,18.75,XY24,NTT,HEALTHCARE
44,JCHAC,2282.00,-1.92,-32.69,48.56,-0.01,41772.0,-41783.0,86022.0,17508.70,9.0,M-SC,0.93,233.0,-1.00,0.62,0.00,OX40N,NTT,AC
50,MASFIN,398.61,-1.83,-3.80,26.87,22.05,25328.0,-3720.0,94260.0,-16.89,53.0,H-SC,6.91,164.0,-0.15,0.68,37.01,XR,ATH,FINANCE
28,GREENPANEL,537.00,-1.45,-35.68,122.55,43.14,144303.0,-65328.0,117750.0,163.57,28.0,M-SC,7.14,240.0,-0.45,0.85,10.76,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.67,-0.57,107.91,106.72,132331.0,-705.0,122631.0,-50.82,38.0,H-SC,1.57,139.0,-0.01,0.88,22.67,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.47,-0.95,36.93,35.63,76661.0,-1989.0,207585.0,-14.97,45.0,H-MC,3.40,119.0,-0.03,1.49,15.56,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.70,-5.39,119.75,107.91,167278.0,-7955.0,139689.0,-22.98,37.0,M-SC,11.28,216.0,-0.05,1.00,0.70,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.36,-7.43,8.05,0.02,13008.0,-12975.0,161584.0,146.65,69.0,M-SC,15.59,234.0,-1.00,1.16,40.55,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.39,-16.38,19.70,0.09,16600.0,-16511.0,84266.0,95.52,40.0,M-SC,2.77,253.0,-0.99,0.61,10.71,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.27,-23.34,48.01,13.47,99255.0,-62928.0,206739.0,-68.51,33.0,H-SC,2.37,158.0,-0.63,1.49,5.12,XY24,NTT,PAINTS
72,TATAELXSI,9161.00,1.05,-24.64,75.20,32.03,74712.0,-32484.0,99351.0,-18.39,46.0,H-SC,7.87,148.0,-0.43,0.71,11.28,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.80,5.29,63.60,72.27,113475.0,8972.0,178420.0,-9.71,69.0,M-LC,3.07,99.0,0.08,1.28,11.8,XR,NTT,IT
37,IEX,219.0,0.07,1.12,46.98,48.63,94990.0,2239.0,202193.0,-32.69,66.0,H-SC,14.72,136.0,0.02,1.45,12.8,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.80,5.29,63.60,72.27,113475.0,8972.0,178420.0,-9.71,69.0,M-LC,3.07,99.0,0.08,1.28,11.80,XR,NTT,IT
37,IEX,219.00,0.07,1.12,46.98,48.63,94990.0,2239.0,202193.0,-32.69,66.0,H-SC,14.72,136.0,0.02,1.45,12.80,XR,NTT,MISC
38,INDIAMART,4810.62,-0.67,-0.57,107.91,106.72,132331.0,-705.0,122631.0,-50.82,38.0,H-SC,1.57,139.0,-0.01,0.88,22.67,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.47,-0.95,36.93,35.63,76661.0,-1989.0,207585.0,-14.97,45.0,H-MC,3.40,119.0,-0.03,1.49,15.56,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.70,-5.39,119.75,107.91,167278.0,-7955.0,139689.0,-22.98,37.0,M-SC,11.28,216.0,-0.05,1.00,0.70,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.02,-20.70,87.21,48.45,138662.0,-41508.0,158998.0,-26.32,13.0,X-MC,9.39,64.0,-0.30,1.14,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.07,-36.99,114.59,35.20,92898.0,-47600.0,81070.0,6.28,26.0,X-SC,14.36,92.0,-0.51,0.58,1.25,X40,NTT,FOOTWEAR
8,AWL,485.00,-0.24,-21.84,96.40,53.50,227347.0,-65914.0,235837.0,-62.23,27.0,X-MC,1.50,58.0,-0.29,1.69,4.75,XY24,NTT,FMCG
22,DIXON,18931.72,-0.86,-10.74,36.22,21.58,45305.0,-15057.0,125082.0,-55.00,27.0,X-MC,5.71,56.0,-0.33,0.90,10.74,X40N,ATH,IT
46,JSWINFRA,342.00,-0.61,-9.54,27.37,15.23,49605.0,-19109.0,181238.0,-22.54,30.0,X-MC,6.80,66.0,-0.39,1.30,19.64,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-0.87,-23.34,114.13,64.15,208185.0,-55541.0,182410.0,-55.00,43.0,X-SC,1.47,82.0,-0.27,1.31,2.32,XY24,BTT,CEMENT
8,AWL,485.0,-0.24,-21.84,96.40,53.50,227347.0,-65914.0,235837.0,-62.23,27.0,X-MC,1.50,58.0,-0.29,1.69,4.75,XY24,NTT,FMCG
43,ITC,452.0,0.65,-1.51,12.13,10.44,23910.0,-3022.0,197116.0,-41.15,45.0,X-LC,1.57,5.0,-0.13,1.42,3.97,X40,NTT,FMCG
15,CAMS,4762.0,1.31,2.46,21.79,24.79,56231.0,6204.0,258060.0,-2.36,52.0,X-SC,2.10,86.0,0.11,1.85,26.17,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.17,1.51,29.02,30.97,52063.0,2666.0,179404.0,-22.41,48.0,X-MC,2.21,75.0,0.05,1.29,14.26,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.02,-20.70,87.21,48.45,138662.0,-41508.0,158998.0,-26.32,13.0,X-MC,9.39,64.0,-0.30,1.14,0.00,X40N,ATH,FINANCE
55,QUESS,424.00,-0.56,-30.51,105.55,42.84,47682.0,-19831.0,45175.0,-54.27,36.0,X-SC,37.30,83.0,-0.42,0.32,0.00,XY24,NTT,MISC
19,COLPAL,3726.84,0.87,-19.50,75.79,41.51,160682.0,-51355.0,212010.0,-4.10,36.0,X-MC,7.36,61.0,-0.32,1.52,0.87,XY25,ATH,FMCG
53,PAGEIND,51605.08,1.13,-8.00,37.03,26.07,55782.0,-13100.0,150640.0,-32.28,34.0,X-MC,10.68,55.0,-0.23,1.08,1.13,X40,ATH,APPARELS
54,PGHH,17907.65,1.22,-4.44,39.98,33.76,76720.0,-8925.0,191895.0,-32.64,39.0,X-MC,4.62,57.0,-0.12,1.38,1.22,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.56,-30.51,105.55,42.84,47682.0,-19831.0,45175.0,-54.27,36.0,X-SC,37.30,83.0,-0.42,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,0.63,-47.07,187.74,52.31,144252.0,-68324.0,76836.0,-43.66,39.0,X-SC,5.33,91.0,-0.47,0.55,2.85,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.07,-36.99,114.59,35.20,92898.0,-47600.0,81070.0,6.28,26.0,X-SC,14.36,92.0,-0.51,0.58,1.25,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.86,-10.74,36.22,21.58,45305.0,-15057.0,125082.0,-55.00,27.0,X-MC,5.71,56.0,-0.33,0.90,10.74,X40N,ATH,IT
51,MEDANTA,1486.00,-1.93,-0.72,24.98,24.08,31483.0,-916.0,126034.0,-8.70,37.0,X-SC,5.96,89.0,-0.03,0.91,18.75,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.75,-12.09,35.68,19.28,108516.0,-41819.0,304137.0,-23.30,70.0,X-LC,6.53,1.0,-0.39,2.19,12.43,X40,ATH,IT
41,INFY,2275.00,1.13,7.47,44.11,54.87,148326.0,23375.0,336263.0,-16.37,64.0,X-LC,2.71,2.0,0.16,2.42,16.37,X40,BTT,IT
75,TMPV,600.00,0.14,-15.32,67.90,42.18,158929.0,-42346.0,234064.0,-24.30,33.0,X-LC,3.54,3.0,-0.27,1.68,1.39,XY24,NTT,AUTO
81,VBL,671.64,1.57,-2.03,38.48,35.67,119069.0,-6412.0,309430.0,-13.74,62.0,X-LC,5.28,4.0,-0.05,2.22,11.54,X40N,ATH,FMCG
43,ITC,452.00,0.65,-1.51,12.13,10.44,23910.0,-3022.0,197116.0,-41.15,45.0,X-LC,1.57,5.0,-0.13,1.42,3.97,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,1.34,-13.94,111.09,81.67,89690.0,-13074.0,80736.0,7255.56,49.0,L-SC,16.49,271.0,-0.15,0.58,56.35,XR,NTT,CERAMICS
67,SONACOMS,806.63,1.70,-13.50,61.33,39.54,53664.0,-13659.0,87500.0,-32.36,56.0,M-SC,6.26,220.0,-0.25,0.63,23.38,AR,ATH,AUTO
50,MASFIN,398.61,-1.83,-3.80,26.87,22.05,25328.0,-3720.0,94260.0,-16.89,53.0,H-SC,6.91,164.0,-0.15,0.68,37.01,XR,ATH,FINANCE
13,BSOFT,831.70,0.86,-22.69,95.05,50.79,103350.0,-31921.0,108732.0,-0.66,73.0,H-SC,5.59,151.0,-0.31,0.78,26.60,XR,ATH,IT
69,SURYODAY,216.00,-0.31,-19.99,53.77,23.03,77041.0,-35792.0,143279.0,55.70,47.0,H-SC,10.09,167.0,-0.46,1.03,42.16,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.36,-7.43,8.05,0.02,13008.0,-12975.0,161584.0,146.65,69.0,M-SC,15.59,234.0,-1.00,1.16,40.55,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.72,3.98,16.31,20.95,42698.0,10023.0,261791.0,2.88,80.0,X-LC,17.97,31.0,0.23,1.88,41.04,X40,ATH,PAINTS
13,BSOFT,831.70,0.86,-22.69,95.05,50.79,103350.0,-31921.0,108732.0,-0.66,73.0,H-SC,5.59,151.0,-0.31,0.78,26.60,XR,ATH,IT
40,INDUSINDBK,1800.00,-0.40,-37.48,112.54,32.89,54326.0,-28934.0,48273.0,-696.43,59.0,L-MC,6.00,259.0,-0.53,0.35,32.94,XR,NTT,BANKS
32,HCLTECH,1922.01,1.40,7.96,15.54,24.73,40586.0,19246.0,261170.0,12.76,68.0,X-LC,5.22,7.0,0.47,1.88,24.40,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.38
1,25,45.21
2,50,76.19


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.26
MC    29.46
LC    24.29
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.98
X40      21.90
X40N     12.14
XR        9.94
AR        9.28
XY25      9.20
OX40N     7.72
SR        1.04
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.42
X-MC    22.86
X-LC    20.81
M-SC    12.10
X-SC     8.33
H-MC     4.85
L-SC     1.41
M-MC     1.40
M-LC     1.28
H-LC     1.20
L-LC     1.00
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.04,-6.07,40.38
IT,12.51,-15.78,78.38
FINANCE,9.49,-17.46,67.09
MISC,7.59,-22.87,73.21
PAINTS,6.15,-7.85,23.86
ELECTRICAL,6.15,-9.78,49.99
INSURANCE,4.45,-2.23,38.25
PHARMA,3.98,-0.82,33.09
AUTO,3.41,-20.97,72.84


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3143271.0,21
AR,1279674.0,10
XR,1278723.0,13
X40,993226.0,14
X40N,757111.0,9
OX40N,717004.0,10
XY25,360243.0,6
SR,276515.0,2
MH,71861.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3580815.0,25
M-SC,1376399.0,15
X-MC,1298919.0,16
X-LC,868111.0,11
X-SC,759593.0,8
H-MC,404027.0,3
L-SC,264285.0,3
M-LC,113475.0,1
H-LC,66738.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1252824.0      6
           AR         879712.0      5
M-SC       XY24       789783.0      6
H-SC       XR         766600.0      7
X-MC       X40        450298.0      7
           XY24       387818.0      3
X-LC       X40        364036.0      5
H-SC       OX40N      333303.0      4
M-SC       OX40N      296948.0      5
X-SC       X40N       293351.0      3
           XY24       287350.0      3
H-SC       SR         276515.0      2
X-MC       X40N       271372.0      4
X-LC       XY24       231815.0      2
H-MC       AR         210346.0      2
           XY24       193681.0      1
X-LC       X40N       192388.0      2
X-MC       XY25       189431.0      2
X-SC       X40        178892.0      2
L-SC       XR         177532.0      2
M-SC       XR         166790.0      2
           AR         122878.0      2
M-LC       XR         113475.0      1
L-SC       OX40N       86753.0      1
X-LC       XY25        79872.0      2
H-SC       MH          71861.0      1
H-LC       AR          66738.0      1
L-MC       XR          54326.0      1
M-MC       XY25        50710.0      1
L-LC       XY25        40230.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
